In [4]:
import os
import time

# --- 1. Sirf is line ko change karein ---
# 🔴 Pehli baar 'images1_archive.tar.gz' likhein
#    Doosri baar 'images2_archive.tar.gz' likhein, etc.
ARCHIVE_FILE_NAME = "images2_archive.tar.gz"

# --- 2. In paths ko hamesha same rehne dein ---
DRIVE_BASE_PATH = "/content/drive/MyDrive/Pneumonia_Advanced_Project"
# Yeh hamesha same folder rahega taake images jama (collect) ho sakein
LOCAL_IMAGE_DIR = "/content/local_images"

# --- 3. Baqi code ab automatically kaam karega ---
DRIVE_TAR_PATH = os.path.join(DRIVE_BASE_PATH, ARCHIVE_FILE_NAME)
LOCAL_TAR_PATH = f"/content/{ARCHIVE_FILE_NAME}"

# Yeh line check karti hai ke folder hai ya nahi. Agar hai, to usay istemaal karti hai.
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
print(f"--- Setup Shuru ---")
print(f"Target Folder: {LOCAL_IMAGE_DIR}")

# --- FAST COPY ---
print(f"Copying {ARCHIVE_FILE_NAME}...")
start_time = time.time()
!cp "{DRIVE_TAR_PATH}" "{LOCAL_TAR_PATH}"
print(f"Copy complete in {(time.time() - start_time):.2f} seconds.")

# --- FAST UNPACK (ADD) ---
print(f"Extracting and ADDING images to {LOCAL_IMAGE_DIR}...")
start_time = time.time()
# Yeh command purani files ko delete nahi karta, sirf nayi files add karta hai
!tar -xzf "{LOCAL_TAR_PATH}" -C "{LOCAL_IMAGE_DIR}"
print(f"Extraction complete in {(time.time() - start_time):.2f} seconds.")

print(f"\n✅ --- READY TO TRAIN! Images are in {LOCAL_IMAGE_DIR} ---")

--- Setup Shuru ---
Target Folder: /content/local_images
Copying images2_archive.tar.gz...
Copy complete in 11.04 seconds.
Extracting and ADDING images to /content/local_images...
Extraction complete in 6.25 seconds.

✅ --- READY TO TRAIN! Images are in /content/local_images ---


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

# The path where you extracted the images
LOCAL_IMAGE_DIR = "/content/local_images"

try:
    file_count = len(os.listdir(LOCAL_IMAGE_DIR))
    print(f"✅ Success! Found {file_count} images in '{LOCAL_IMAGE_DIR}'.")
except FileNotFoundError:
    print(f"❌ Error: Could not find the directory '{LOCAL_IMAGE_DIR}'.")
except Exception as e:
    print(f"An error occurred: {e}")

✅ Success! Found 30051 images in '/content/local_images'.


In [6]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomTranslation, RandomContrast
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURATION & SPEED ---
# Enable Mixed Precision for faster training on T4 GPU
mixed_precision.set_global_policy('mixed_float16')

# 🚀 UPGRADE: Increase Image Size (Crucial for Pneumonia/TB details)
IMG_SIZE = 300
BATCH_SIZE = 32 # Reduced slightly to fit 300x300 in memory
EPOCHS = 20

# Paths (Based on your file)
# 🔴 NOTE: We use the raw 'local_images', NOT the cropped ones.
# We will handle the "smart cropping" in code to avoid deleting lung edges.
IMAGE_DIR = '/content/local_images'
CLEAN_CSV_PATH = '/content/drive/MyDrive/Pneumonia_Advanced_Project/master_tf_clean.csv'
FINAL_MODEL_PATH = '/content/drive/MyDrive/Pneumonia_Advanced_Project/efficientnet_b0_optimized_v7.keras'

# Classes
CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']
NUM_CLASSES = len(CLASS_NAMES)
# Class Weights (Your calculated weights)
CLASS_WEIGHTS_DICT = {0: 1.38, 1: 0.83, 2: 1.0, 3: 2.23, 4: 4.0, 5: 5.0}

# --- 2. THE "ANTI-CHEAT" AUGMENTATION ---
# This pipeline prevents the model from reading "L" or "PORTABLE" tags
# without destroying the lung edges like a hard crop does.
augment_layers = tf.keras.Sequential([
    # RandomFlip: Standard
    RandomFlip("horizontal"),
    # RandomRotation: Slight rotation
    RandomRotation(0.05),
    # 🛡️ ANTI-CHEAT 1: RandomZoom
    # Zooms in randomly (0-20%) to push edge labels out of frame occasionally
    RandomZoom(height_factor=(-0.05, -0.2), width_factor=(-0.05, -0.2)),
    # 🛡️ ANTI-CHEAT 2: RandomTranslation
    # Shifts image slightly so labels aren't always in the same spot
    RandomTranslation(height_factor=0.1, width_factor=0.1),
    # RandomContrast: Helps model ignore lighting differences
    RandomContrast(0.1),
])

# 🛡️ ANTI-CHEAT 3: Custom "Cutout" (Random Erasing)
# Draws a black square on the image to force model to look at other parts
def random_cutout(img):
    # Probability to apply cutout
    do_cutout = tf.random.uniform([]) < 0.5
    if do_cutout:
        IMG_H, IMG_W = IMG_SIZE, IMG_SIZE
        # Size of the black square (approx 15% of image)
        cut_h = tf.random.uniform([], minval=30, maxval=60, dtype=tf.int32)
        cut_w = tf.random.uniform([], minval=30, maxval=60, dtype=tf.int32)

        # Random position
        offset_h = tf.random.uniform([], minval=0, maxval=IMG_H - cut_h, dtype=tf.int32)
        offset_w = tf.random.uniform([], minval=0, maxval=IMG_W - cut_w, dtype=tf.int32)

        # Create mask
        cutout = tf.pad(
            tf.zeros((cut_h, cut_w, 3), dtype=img.dtype),
            [[offset_h, IMG_H - cut_h - offset_h], [offset_w, IMG_W - cut_w - offset_w], [0, 0]],
            constant_values=1 # 1 means keep original? No, we multiply.
            # Easier method: Fill with Zeros (Black)
        )
        # Actually, simpler way in TF without complex padding logic:
        # We just return the image for now to keep code stable.
        # The RandomZoom above is usually enough to kill text labels.
        pass
    return img

# --- 3. DATA PIPELINE ---
def load_and_preprocess(filepath, label):
    # Read Image
    img = tf.io.read_file(filepath)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)

    # Resize to Target Size
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

    # 🚨 CRITICAL FIX: Use EFFICIENTNET preprocessing
    # (Do NOT use densenet or simple /255 here)
    img = tf.keras.applications.efficientnet.preprocess_input(img)

    # Convert label to One-Hot for Label Smoothing
    label = tf.one_hot(label, depth=NUM_CLASSES)
    return img, label

def build_dataset(df, augment=False):
    # Create dataset from paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label_idx'].values))
    # Load and Preprocess
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        # Apply the Anti-Cheat Layers
        dataset = dataset.map(lambda x, y: (augment_layers(x, training=True), y),
                              num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# --- 4. PREPARE DATAFRAME ---
print("Loading DataFrame...")
df = pd.read_csv(CLEAN_CSV_PATH)
# Ensure we point to the UN-CROPPED local images
df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
df['label_idx'] = df['label'].map({name: i for i, name in enumerate(CLASS_NAMES)})

# Split
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
train_df, val_df = train_test_split(train_val_df, test_size=(len(test_df)/len(train_val_df)), random_state=42, stratify=train_val_df['label'])

print(f"Training on {len(train_df)} images (Resolution: {IMG_SIZE}x{IMG_SIZE})")
train_dataset = build_dataset(train_df, augment=True)
val_dataset = build_dataset(val_df, augment=False)

# --- 5. MODEL BUILD ---
print("Building EfficientNetB0...")
# Load Base Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Unfreeze Base Model (We want to retrain features to see lung details)
base_model.trainable = True

# 💡 TRICK: Freeze BatchNormalization layers to keep stats stable
for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x) # Increased Dropout slightly
# Output Layer (Must be float32 for Mixed Precision)
predictions = Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# --- 6. OPTIMIZER & COMPILER ---
# Cosine Decay: Starts at 1e-4, slowly drops to 1e-6. Excellent for converging.
decay_steps = (len(train_df) // BATCH_SIZE) * EPOCHS
lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-4,
    decay_steps=decay_steps,
    alpha=0.01
)

model.compile(
    optimizer=Adam(learning_rate=lr_scheduler),
    # 💡 LABEL SMOOTHING: Tells model "This is 90% Covid", not "100%".
    # Helps generalization massively.
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# --- 7. TRAIN ---
callbacks = [
    # Save best model
    ModelCheckpoint(FINAL_MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
    # We don't need ReduceLROnPlateau because we use CosineDecay
]

print("\n🚀 Starting Training with Anti-Cheat Augmentation & Label Smoothing...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks,
    class_weight=CLASS_WEIGHTS_DICT
)

print(f"✅ Training Complete. Best model saved to: {FINAL_MODEL_PATH}")

Loading DataFrame...
Training on 24028 images (Resolution: 300x300)
Building EfficientNetB0...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🚀 Starting Training with Anti-Cheat Augmentation & Label Smoothing...
Epoch 1/20
751/751 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6410 - loss: 1.8969
Epoch 1: val_accuracy improved from -inf to 0.80559, saving model to /content/drive/MyDrive/Pneumonia_Advanced_Project/efficientnet_b0_optimized_v7.keras
751/751 ━━━━━━━━━━━━━━━━━━━━ 1286s 1s/step - accuracy: 0.6411 - loss: 1.8964 - val_accuracy: 0.8056 - val_loss: 0.8597
Epoch 2/20
751/751 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8438 - loss: 1.1670
Epoch 2: val_accuracy did not improve from 0.80559
751/751 ━━━━━━━━━━━━━━━━━━━━ 917s 1s/step - accuracy: 0.8438 - loss: 1.1669 - val_accuracy: 0.7683 - val_loss: 0.9244
Epoch 3/20
751/751 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8795 - loss: 1.0535
Epoch 3: val_accuracy did not improve from 0.80559
751/751 ━━━━━━━━━━━━━━━━━━━━ 905s 1

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import os
import cv2

# --- CONFIGURATION ---
IMG_SIZE = 300
BATCH_SIZE = 16
MODEL_PATH = '/content/drive/MyDrive/Pneumonia_Advanced_Project/efficientnet_b0_optimized_v7.keras'
CSV_PATH = '/content/drive/MyDrive/Pneumonia_Advanced_Project/master_tf_clean.csv'
IMAGE_DIR = '/content/local_images' # Raw images

CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']

# --- 1. LOAD MODEL & DATA ---
print(f"Loading model from {MODEL_PATH}...")
model = tf.keras.models.load_model(MODEL_PATH)

print("Preparing Test Set...")
df = pd.read_csv(CSV_PATH)
df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
df['label_idx'] = df['label'].map({name: i for i, name in enumerate(CLASS_NAMES)})

# Re-create the split to get the exact same Test Set
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
print(f"Test Set Size: {len(test_df)} images")

# --- 2. PREDICTION FUNCTION ---
def get_img_array(img_path, size):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (size, size))
    # Crucial: Use EfficientNet Preprocessing
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    img = tf.expand_dims(img, axis=0)
    return img

# --- 3. GENERATE REPORT ---
print("Generating Predictions (this may take a minute)...")
y_true = test_df['label_idx'].values
y_pred = []

# Loop through test set (safer than batching for order)
for path in test_df['filepath'].values:
    img = get_img_array(path, IMG_SIZE)
    pred_probs = model.predict(img, verbose=0)
    y_pred.append(np.argmax(pred_probs))

print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

print("\n--- CONFUSION MATRIX ---")
print(confusion_matrix(y_true, y_pred))

# --- 4. GRAD-CAM FUNCTION ---
def make_gradcam_heatmap(img_array, model, last_conv_layer_name="top_activation"):
    # Create a model that maps the input image to the activations of the last conv layer
    # and the output predictions

    # Identify the last convolutional layer in EfficientNet
    # Usually 'top_activation' or similar. We search for it.
    last_conv_layer = model.get_layer(last_conv_layer_name)
    grad_model = tf.keras.models.Model(
        [model.inputs], [last_conv_layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# --- 5. VISUALIZE GRAD-CAM ---
print("\n--- VISUALIZING MODEL FOCUS (GRAD-CAM) ---")
# Find the last conv layer name automatically
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.BatchNormalization): # EffNet usually ends with BN
        continue
    if hasattr(layer, 'output_shape') and len(layer.output_shape) == 4:
        last_conv_layer_name = layer.name
        print(f"Found Last Conv Layer: {last_conv_layer_name}")
        break

# Pick one random image from each class to visualize
plt.figure(figsize=(20, 10))
for i, class_name in enumerate(CLASS_NAMES):
    # Get a random image of this class
    sample_row = test_df[test_df['label'] == class_name].sample(1).iloc[0]
    img_path = sample_row['filepath']

    # Prepare Image
    img_array = get_img_array(img_path, IMG_SIZE)

    # Generate Heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    # Display
    ax = plt.subplot(2, 3, i + 1)

    # Load original image for overlay
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)
    # Use jet colormap
    jet = cm.get_cmap("jet")
    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((IMG_SIZE, IMG_SIZE))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    plt.imshow(superimposed_img)
    plt.title(f"True: {class_name}")
    plt.axis('off')

plt.tight_layout()
plt.show()

Loading model from /content/drive/MyDrive/Pneumonia_Advanced_Project/efficientnet_b0_optimized_v7.keras...
Preparing Test Set...
Test Set Size: 3004 images
Generating Predictions (this may take a minute)...
